In [1]:
import json,re,os,datetime,time,string
from pathlib import Path
import requests
from langchain.llms.base import LLM
from typing import *
from deepeval.models.base_model import DeepEvalBaseLLM
import traceback,random
import pandas as pd
import numpy as np
import evalplus.sanitize,evalplus.syncheck
from collections import defaultdict
from typing import *

d:\Anaconda\envs\vllm\lib\site-packages\deepeval\__init__.py:41: UserWarning: You are using deepeval version 0.21.27, however version 0.21.39 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [ ]:
INFINI_API_List = ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]

In [ ]:
modelProfileDict = {
    'llama-3-70b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-70b-Instruct是此系列里，700亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-70b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年12月。",
    'llama-3-8b-instruct':"Llama3系列是由Meta开发的Llama系列全新的第三代版本，包含一系列预训练和指令调优的文本生成式模型。Llama3基于优化后的Transformer架构，预训练过程中使用了超过15T tokens的数据，调优后的模型使用SFT和RLHF，以更好地贴合人类对可用性和安全性的偏好。Llama3-8b-Instruct是此系列里，80亿参数的指令调优的模型，针对对话场景用例进行了优化，并在常见的行业基准测试中超越了许多可用的开源聊天模型。Llama3-8b-Instruct支持模型上下文至8k tokens，该模型的数据的知识截止日期为2023年3月。",
    'chatglm3':"ChatGLM3是智谱AI与清华KEG实验室发布的闭源模型，支持 8K 上下文，经过海量中英标识符的预训练与人类偏好对齐训练，相比一代模型在 MMLU、C-Eval、GSM8K 分别取得了16%、36%、280%的提升，并登顶中文任务榜单C-Eval。适用于对知识量、推理能力、创造力要求较高的场景，比如广告文案、小说写作、知识类写作、代码生成等。",
    'chatglm2-6b':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。在保留了初代模型对话流畅、部署门槛较低等众多优秀特性的基础之上，在 MMLU、C-Eval、GSM8K、BBH等主流学术数据集上，都得到了显著的性能提升，并通过基于 FlashAttention 技术，将对话模型的上下文长度（Context Length）提升至 8k tokens，允许更多轮次的对话。",
    'chatglm2-6b-32k':"ChatGLM2-6b 是由智谱开发的 ChatGLM 系列的第二代版本，支持中英双语的60亿参数规模的开源模型。相较于ChatGLM2-6B，ChatGLM2-6b-32k支持更长的模型上下文至32k tokens。",
    'infini-megrez-7b':"由无问芯穹公司自主研发的70亿参数大语言模型。在逻辑推理、对话能力等方面有优秀的性能表现。配合无问芯穹自研高效推理引擎，同时支持Nvidia和AMD的GPU，具备更快的推理速度，在性能表现方面更上一层楼。",
    'llama-2-7b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-13b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-7b-chat是其中70亿的主流参数大小的模型，适用于chat场景，更擅长英文相关的内容。模型支持 4k tokens上下文。",
    'llama-2-70b-chat':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-chat是其中700亿参数的大模型，适用于chat场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'llama-2-70b':"Llama2是由Meta开发并开源的大型语言模型（LLM）系列，这是一组从70亿到700亿参数不同规模、经过预训练和微调的生成式文本模型。架构层面，Llama2是一个使用优化型转换器架构的自动回归语言模型。调整后的版本使用有监督的微调（SFT）和带有人类反馈的强化学习（RLHF）以对齐人类对有用性和安全性的偏好。Llama2较Llama系列在多种学术数据集上有着更加不俗的表现，为大量其他模型提供了设计和开发的思路。Llama2-70b-base是其中700亿参数的基础大模型，适用于通用语言任务场景，更擅长英文相关的内容，相较该系列里其他规模的的模型，有更强的综合能力。模型支持 4k tokens上下文。",
    'baichuan2-7b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-7b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持4k tokens上下文。",
    'baichuan2-13b-chat':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-chat是130亿参数规模用于对话的模型，在C-Eval、MMLU、CMMLU等主流评测数据集上都有不俗的表现。该基模型支持8k tokens上下文。",
    'baichuan2-13b-base':"Baichuan 2 是百川智能推出的新一代开源大语言模型，采用 2.6 万亿 Tokens 的高质量语料训练。Baichuan2-13b-base是130亿参数规模的基础模型，适用于通用对话和文本续写，较chat模型更适合于复杂场景的微调后使用。该基模型支持4k tokens上下文。",
    'chatglm3-6b':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。ChatGLM3采用了全新设计的 Prompt 格式，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。模型支持 8k tokens上下文。",
    'chatglm3-6b-32k':"ChatGLM3-6b 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源模型。相较于ChatGLM之前系列的模型，ChatGLM3采用了更多样的训练数据，并原生支持工具调用（Function Call）、代码执行（Code Interpreter）和 Agent 任务等复杂场景。ChatGLM3-6b-32k在ChatGLM3-6b 基础上进一步强化了对于长文本的理解能力，能够更好的处理最多32k tokens长度的上下文。",
    'chatglm3-6b-base':"ChatGLM3-6b-base 是由智谱开发的 ChatGLM 系列最新一代的60亿参数规模的开源的基础模型。ChatGLM3-6B-Base 采用了更多样的训练数据、更充分的训练步数和更合理的训练策略。基础模型更适合于复杂场景的微调后使用，该基模型支持32k tokens上下文。",
    'qwen-7b-chat':"通义千问-7B-chat（Qwen-7B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的70亿参数规模的大语言模型。相较于Qwen-7B-Base模型，Qwen-7B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-14b-chat':"通义千问-14B-chat（Qwen-14B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的140亿参数规模的大语言模型。相较于Qwen-14B-Base模型，Qwen-14B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 8k tokens上下文。",
    'qwen-72b-chat':"通义千问-72B-chat（Qwen-72B-chat）是阿里云研发的基于Transformer，在超大规模的预训练数据上进行训练得到的720亿参数规模的大语言模型。相较于Qwen-72B-Base模型，Qwen-72B-chat是针对于对话场景以及一些常见的智能对话需求指令对齐的AI助手模型，在更多文本相关的问答场景上有更好的指令跟随能力。模型支持 32k tokens上下文。",
    'qwen-72b':"通义千问-72B（Qwen-72B）是阿里云研发的通义千问大模型系列的720亿参数规模的模型。Qwen-72B是基于Transformer的大语言模型, 在超大规模的预训练数据上进行训练得到。预训练数据类型多样，覆盖广泛，包括大量网络文本、专业书籍、代码等。模型支持 32k tokens上下文。",
    'qwen1.5-7b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-7b-chat是其中专用于chat场景的70亿参数的主流大小模型。",
    'qwen1.5-14b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-14b-chat是其中专用于chat场景的140亿参数的主流大小模型。",
    'qwen1.5-72b-chat':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-chat是其中专用于chat场景的720亿参数的大模型。",
    'qwen1.5-72b':"Qwen1.5系列是Qwen2的Beta版本，是一个基于Transformer的仅解码语言模型，在海量数据上进行预训练。与之前发布的Qwen系列版本相比，Qwen1.5系列base与chat模型均能支持多种语言，在整体聊天和基础能力上都得到了提升，并且支持32k tokens上下文。Qwen1.5-72b-base是其中的720亿参数的基础大模型，适合多种场景的使用。",
}
answerModelList = [    
    'llama-3-8b-instruct',
    # 'chatglm3',
    # 'chatglm2-6b',
    # 'chatglm2-6b-32k',
    'infini-megrez-7b',
    'llama-2-7b-chat',
    'llama-2-13b-chat',
    'llama-2-70b-chat',
    'llama-2-70b',
    'baichuan2-7b-chat',
    'baichuan2-13b-chat',
    'baichuan2-13b-base',
    # 'chatglm3-6b',
    # 'chatglm3-6b-32k',
    # 'chatglm3-6b-base',
    'qwen-7b-chat',
    'qwen-14b-chat',
    'qwen-72b-chat',
    'qwen-72b',
    'qwen1.5-7b-chat',
    'qwen1.5-14b-chat',
    'qwen1.5-72b',
]

In [ ]:
def LLMCompletions(prompt,modelName:str = "infini-megrez-7b",INFINI_API = "sk-c7cssl4bkglsrwf2",returnContent:bool = True,**kwargs):
    url = "https://cloud.infini-ai.com/maas/"+modelName+"/nvidia/chat/completions"
    payload = {
        "model": "string",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "temperature": kwargs["temperature"] if "temperature" in kwargs else 0.7,
        "top_p": kwargs["top_p"] if 'top_p' in kwargs else 1,
        "top_k": kwargs['top_k'] if 'top_k' in kwargs else -1,
        "n": kwargs['n'] if 'n' in kwargs else 1,
        "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else None,
        "stop": kwargs['stop'] if 'stop' in kwargs else None,
        "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else 0,
        "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else 0
    }
    headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+INFINI_API,
    } 
    response = requests.post(url, json=payload, headers=headers,)
    if response.status_code == 200:
        response.encoding = 'utf-8'
        data = response.json()
        content = data['choices'][0]['message']['content']
        
        content = content.replace(',\n}','\n}')
        if returnContent:
            return content
        try:
            content = json.loads(content)
        except:
            content = content.replace('\n','')
        data['choices'][0]['message']['content'] = content
        
        return json.dumps(data['choices'][0]['message']['content'])

    else:
        print(response.status_code)
        return "Cannot connect to the model "+modelName

In [ ]:
data_format_ins = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'retrieval_context':None,
    'time':-1
}
def joinErrorToData(errorFile,saveFile):
    with open(errorFile, 'r') as ef:
        data_ef = json.load(ef)
    fileName = data_ef['fileName']
    with open(saveFile) as sf:
        data_sf = json.load(sf)
    if fileName != data_sf['fileName']:
        print('FileName not match')
        return 
    if not data_ef['data']:
        print('The Errors of this ErrorFile all have been solved!')
        return 
    errorItem = []
    for item in data_ef['data']:
        data_format_ins['id'] = item['id']
        model = item['AnswerModel']
        data_format_ins['AnswerModel'] = model
        prompt = item['input']
        data_format_ins['input'] = prompt
        data_format_ins['expected_output'] = item['expected_output']
        start = time.perf_counter_ns()
        actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
        end = time.perf_counter_ns()
        delta = end-start
        idx = 0
        while actual_output == "Cannot connect to the model "+model and idx<2:
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model,INFINI_API=INFINI_API_2)
            end = time.perf_counter_ns()
            delta = end-start
            idx += 1
        if actual_output == "Cannot connect to the model "+model:
            errorItem.append(item)
            print("[error]:\t"+str(errorItem[-1]))
            continue
        print(actual_output)
        data_format_ins['time'] = delta
        data_format_ins['actual_output'] = actual_output
        data_sf['data'].append(data_format_ins.copy())
    data_ef['data'] = errorItem
    with open(saveFile,'w') as saveF:
        json.dump(data_sf,saveF)
    with open(errorFile,'w') as error:
        json.dump(data_ef,error)
    if errorItem:
        print("There are still some errors! ")
    else:
        print('The Errors of this ErrorFile all have been solved!')

In [ ]:
def removeDeprecatedModel(filePath):
    with open(filePath) as f:
        data = json.load(f)
    new_data = []
    for item in data['data']:
        if item['AnswerModel']  in answerModelList:
            new_data.append(item)
    data['data'] = new_data[:]
    with open(filePath,'w') as f:
        json.dump(data,f)

In [ ]:
# removeDeprecatedModel('./data/1-2_1_low_freq_ent_sample.json')

In [ ]:
# joinErrorToData('./data/4-2_r_with_triples_sampleError.json','./data/4-2_r_with_triples_sample.json')

In [ ]:
# diff = collections.Counter()

In [ ]:
# qapairs = []
# model_input_id = []
# with open('./data/1-1_2_high_freq_ent_sample.json') as saveF:
#     data = json.load(saveF)
# with open('E:/Repository/KoLA/Sample_Data/1-1_2_high_freq_ent_sample.json') as file:
#     data_o = json.load(file)
# instructions = data_o['adapter_spec']['instructions']
# for item in data_o['request_states']:
#     qapairs.append((item['instance']['id'],instructions+'\n'+item['instance']['input']['text']))
# for item in data['data']:
#     model_input_id.append([item['AnswerModel'],item['input'],item['id']])
# check = [ False for i in range(len(model_input_id))]
# for model in answerModelList:
#     for id,text in qapairs:
#         for i,item in enumerate(model_input_id):
#             if model == item[0] and text == item[1] and id == item[2]:
#                 check[i] = True

In [ ]:
# for model,text,id in model_input_id:
#     diff[model] += 1
#     diff[text] += 1
#     diff[id] += 1

In [ ]:
### deprecated ChatLLM,CustomLLM
# class ChatLLM(LLM):
#     modelName = "qwen1.5-72b-chat"
#     INFINI_API = "sk-c7cssl4bkglsrwf2"
#     temperature = 0.8
#     top_p = 1
#     top_k = -1
#     n = 1
#     max_tokens:int = None
#     stop:Optional[List[str]] = None
#     presence_penalty = 0
#     frequency_penalty = 0
    
#     headers = {
#             'Content-Type': "application/json",
#             'Accept': "*/*",
#             'Authorization': "Bearer "+INFINI_API,
#     }   
#     @property
#     def _llm_type(self)->str:
#         return "ChatLLM"
#     @property
#     def _identifying_params(self)->Mapping[str,Any]:
#         _param_dict = {
#             "modelName":ChatLLM().modelName,
#             "INFINI_API":ChatLLM().INFINI_API,
#             "stream":bool(ChatLLM().stream),
#             "temperature":ChatLLM().temperature,
#             "top_p":ChatLLM().top_p,
#             "top_k":ChatLLM().top_k,
#             "n":ChatLLM().n,
#             "max_tokens":ChatLLM().max_tokens,
#             "stop":ChatLLM().stop,
#             "presence_penalty":ChatLLM().presence_penalty,
#             "frequency_penalty":ChatLLM().frequency_penalty,
#         }
#         return _param_dict
#     @classmethod  
#     def _call(self, prompt: str, stop: Optional[List[str]]= None,  **kwargs: Any) -> str:
#         url = "https://cloud.infini-ai.com/maas/"+ChatLLM().modelName+"/nvidia/chat/completions"
#         payload = {
#             "model": "string",
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": prompt
#                 }
#             ],
#             "stream": False,
#             "temperature": kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature,
#             "top_p": kwargs["top_p"] if 'top_p' in kwargs else ChatLLM().top_p,
#             "top_k": kwargs['top_k'] if 'top_k' in kwargs else ChatLLM().top_k,
#             "n": kwargs['n'] if 'n' in kwargs else ChatLLM().n,
#             "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens,
#             "stop": stop if stop else ChatLLM().stop,
#             "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty,
#             "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty
#         }
#         response = requests.post(url, json=payload, headers=ChatLLM().headers)
#         if response.status_code == 200:
#             response.encoding = 'utf-8'
#             data = response.json()
#             content = data['choices'][0]['message']['content']
#             if isinstance(content,str):    
#                 content = content.replace(',\n}','\n}')
#                 content = content.replace(']\n}',']}')
#                 content = content.replace('\\','\\\\')
#                 flag = False
#                 if 'statements' in content:
#                     regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
#                     flag = True
#                 elif 'verdicts' in content:
#                     regex = re.compile('\"verdicts\":\s+\[.*\]\}', re.DOTALL) 
#                     flag = True
#                 if flag:
#                     matchStr =regex.search(content)
#                     if matchStr:
#                         content = '{'+matchStr.group()
#             try:
#                 content = json.loads(content)
#             except:
#                 pass
#             data['choices'][0]['message']['content'] = content
                
#             return json.dumps(data['choices'][0]['message']['content'])

#         else:
#             return "Cannot connect to the model "+ChatLLM().modelName
#     def setParameter(self,**kwargs):
#         self.temperature = kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature
#         self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else ChatLLM().top_p
#         self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else ChatLLM().top_k
#         self.n = kwargs['n'] if 'n' in kwargs else ChatLLM().n
#         self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens
#         self.stop = kwargs['stop'] if 'stop' in kwargs else ChatLLM().stop
#         self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty
#         self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty

# class CustomLLM(DeepEvalBaseLLM):
#     def __init__(
#         self,
#         model
#     ):
#         self.model = model

#     def load_model(self):
#         return self.model

#     def generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         return chat_model.invoke(prompt)
#     async def a_generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         res = await chat_model.ainvoke(prompt)
#         return res

#     def get_model_name(self):
#         return "CustomLLM"

# custom_model = ChatLLM()

# evaluateModel = CustomLLM(model=custom_model)

In [ ]:
path = []

In [ ]:
class ChatLLM(LLM):
    @property
    def modelName(self)->str:
        return "qwen1.5-72b-chat"
    @property
    def INFINI_API_List(self)->List[str]:
        return ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]
    @property
    def temperature(self)->float:
        return 0.7
    @property
    def top_p(self)->float:
        return 0.1
    @property
    def top_k(self)->int:
        return -1
    @property
    def n(self)->int:
        return 1
    @property
    def max_tokens(self)->int:
        return None
    @property
    def stop(self)->Optional[List[str]]:
        return None
    @property
    def presence_penalty(self)->float:
        return 0
    @property
    def frequency_penalty(self)->float:
        return 0
    def getHeader(self,index_api):  
        headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+self.INFINI_API_List[index_api%len(self.INFINI_API_List)],
        }
        return headers
    @property
    def _llm_type(self)->str:
        return "ChatLLM"
    @property
    def _identifying_params(self)->Mapping[str,Any]:
        _param_dict = {
            "modelName":self.modelName,
            "INFINI_API":self.getHeader(self.__fields__['index_api'] if 'index_api' in self.__fields__ else 0),
            "stream":bool(self.stream),
            "temperature":self.temperature,
            "top_p":self.top_p,
            "top_k":self.top_k,
            "n":self.n,
            "max_tokens":self.max_tokens,
            "stop":self.stop,
            "presence_penalty":self.presence_penalty,
            "frequency_penalty":self.frequency_penalty,
        }
        return _param_dict

    def _call(self, prompt: str, stop: Optional[List[str]]= None, run_manager= None,**kwargs: Any) -> str:
        url = "https://cloud.infini-ai.com/maas/"+str(self.modelName)+"/nvidia/chat/completions"
        payload = {
            "model": "string",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False,
            "temperature": kwargs["temperature"] if "temperature" in kwargs else self.temperature,
            "top_p": kwargs["top_p"] if 'top_p' in kwargs else self.top_p,
            "top_k": kwargs['top_k'] if 'top_k' in kwargs else self.top_k,
            "n": kwargs['n'] if 'n' in kwargs else self.n,
            "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens,
            "stop": stop if stop else self.stop,
            "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty,
            "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
        }
        index = 0
        if 'index_api' not in self.__fields__:
            self.__fields__['index_api'] = -1
        index_api = self.__fields__['index_api']+1
        length = len(self.INFINI_API_List)
        while index < length:
            response = requests.post(url, json=payload, headers=self.getHeader(index_api))
            if response.status_code == 200:
                response.encoding = 'utf-8'
                data = response.json()
                print("response json success")
                content = data['choices'][0]['message']['content']
                if isinstance(content,str):   
                    content = content.replace(',\n}','\n}')
                    content = content.replace(']\n}',']}')
                    if 'statements' in content:
                        regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
                        matchStr =regex.search(content)
                        if matchStr:
                            content = '{'+matchStr.group()
                    elif 'verdicts' in content:
                        regex = re.compile('\"verdicts\":\s+\[.*\]\}',re.DOTALL)
                        matchStr =regex.search(content)
                        if matchStr is not None:
                            content ='{' +matchStr.group()
                            regex = re.compile("\"reason\":(.*?)\}",re.DOTALL)
                            matchStr = regex.findall(content)
                            for string in matchStr:
                                tmp = string.strip()[1:-1].replace('"','\\\"')
                                tmp = '\"'+tmp+"\""
                                content = content.replace(string,tmp)
                if isinstance(content,str):
                    return content
                data['choices'][0]['message']['content'] = content
                return json.dumps(data['choices'][0]['message']['content'])

            index += 1
            index_api =  (index_api+1)%length
            self.__fields__['index_api'] = index_api
            print(response.status_code)
            try:
                print(response.json())
            except:
                pass
            time.sleep(1)
        return "Cannot connect to the model "+self.modelName
    def setParameter(self,**kwargs):
        self.temperature = kwargs["temperature"] if "temperature" in kwargs else self.temperature
        self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else self.top_p
        self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else self.top_k
        self.n = kwargs['n'] if 'n' in kwargs else self.n
        self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens
        self.stop = kwargs['stop'] if 'stop' in kwargs else self.stop
        self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty
        self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
    

class CustomLLM(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model
    def load_model(self):
        return self.model
    def generate(self, prompt: str) -> str:
        # global path
        chat_model = self.load_model()
        ret = chat_model.invoke(prompt)
        idx = 0
        while ret == "Cannot connect to the model "+self.get_model_name() and idx<5:
            time.sleep(5)
            ret = chat_model.invoke(prompt)
            idx += 1
        # print(ret)
        # path.append(ret)
        return ret
    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        try:
            return self.model.modelName
        except:
            return "CustomLLM"
custom_model = ChatLLM()
evaluateModel = CustomLLM(model=custom_model)

In [ ]:
content = "\"verdicts\": [{\"verdict\":\"yes\"} , {\"verdict\":\"no\",\"reason\":\"123\"}]"
regex = re.compile("\{\s*\"verdict\"\s*:.*?\}")
matchStr = regex.findall(content)
matchStr

In [ ]:
metrics_format_example = {
    'metric_metadata':{
        'metric':None,
        'threshold':0,
        'success':True,
        'score':0.8,
        'reason':'',
        'strictMode': False,
        'evaluationModel': 'CustomLLM',
        'evaluationCost': 0
    },
    'metric_configuration': {
        'threshold': 0.5,
        'evaluation_model': 'CustomLLM',
        'strict_mode': False,
        'include_reason': True
    }
}

data_format_example = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'retrieval_context':None,
    'cached_metrics_data':[
        {
            'metric_metadata':{
                'metric':None,
                'success':True,
                'score':0.8,
                'reason':'',
                'statements':'',
                'verdicts':'',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        },
        metrics_format_example
    ]
}

        

In [ ]:
for dirName,subDirName ,fileNames in os.walk('./data'):
    print(fileNames)

In [ ]:
fileNames = ['1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json']

In [ ]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric,FaithfulnessMetric,HallucinationMetric,BaseMetric
from deepeval.test_case import LLMTestCase

In [ ]:
metrics  = [
    AnswerRelevancyMetric(
        threshold=0.5,
        model=evaluateModel,
        include_reason=True
    ),
    HallucinationMetric(
        threshold=0.5,
        model=evaluateModel,
        include_reason=True
    )
]


In [ ]:
# with open('./data/KoLA/eval/error/2-1_COPEN++csj_sample.json') as f:
#     data = json.load(f)
# data['data'][0]

In [ ]:
# test_case = LLMTestCase(input=data['data'][0]['input'],
#                         actual_output=data['data'][0]['actual_output'],
#                         context=[data['data'][0]['expected_output']])

In [ ]:
# metrics[0].measure(test_case)

In [ ]:
def evaluate_file(filename:Union[str,Path],save_file:Union[str,Path],error_file:Union[str,Path],force_save:bool = False,metrics:List[BaseMetric] = [AnswerRelevancyMetric(threshold=0.5,model=evaluateModel,include_reason=True),HallucinationMetric(threshold=0.5,model=evaluateModel,include_reason=True)]):
    """_summary_
        the function is used to evaluate the LLM output saved in `filename` by the metric in `metrics`,the successful eval results will be saved into `save_file` and the error item will be saved into `error_file`
    
    Args:
        `filename` (Union[str,Path]): the filename saves the LLM generation results
        `save_file` (Union[str,Path]): the filename will save the evaluate results
        `error_file` (Union[str,Path]): the filename will save the error eval item
        `force_save` (bool, optional): if the value is `True`,function will rerun all eval item in `filename` and directly override the `save_file` and `error_file`. 
                    Defaults to False.
        `metrics` (List[BaseMetric], optional): a list of evaluation metrics. 
                    Defaults to [AnswerRelevancyMetric(threshold=0.5,model=evaluateModel,include_reason=True),HallucinationMetric(threshold=0.5,model=evaluateModel,include_reason=True)].
    """
    def is_same_eval_item(x,item):
        if x['id'] == item['id']  and x['AnswerModel'] == item['AnswerModel'] :
            return True
        return False
    with open(filename,'r') as f:
        data = json.load(f)
    save,error = dict(),dict()
    if  Path(save_file).is_file() and not force_save:
        with open(save_file) as f:
            save = json.load(f)
        if 'fileName' in save and  save['fileName'] != data['fileName']:
            print("The save_file does not match the file name!")
            return
    else:
        save = {'fileName':data['fileName'],'class':data['class'],'data':[]}

    if Path(error_file).is_file() and not force_save:
        with open(error_file) as f:
            error = json.load(f)
        if 'fileName' in error and  error['fileName'] != data['fileName']:
            print("The error_file does not match the file name!")
            return
    else:
        error = {'fileName' :data['fileName'],'class':data['class'],'data':[]}
    for metric in metrics:
        for item in data['data']:
            data_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'retrieval_context':None,
                'cached_metrics_data':[]
            }
            metrics_format_ins = {
                'metric_metadata':{
                    'metric':None,
                    'success':True,
                    'score':0.8,
                    'reason':'',
                    'statements':'',
                    'verdicts':'',
                    'evaluationCost': 0
                },
                'metric_configuration': {
                    'threshold': 0.5,
                    'evaluation_model': 'CustomLLM',
                    'strict_mode': False,
                    'include_reason': True
                }
            }
            errors_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'retrieval_context':None,
                'cached_metrics_data':[
                    {            
                        'metric_metadata':{
                            'metric':None,
                        },
                        'metric_configuration': {
                            'threshold': 0.5,
                            'evaluation_model': 'CustomLLM',
                            'strict_mode': False,
                            'include_reason': True
                        }
                    }
                ]
            }
            
            data_format_ins['id'] = item['id']
            data_format_ins['AnswerModel'] = item['AnswerModel']
            data_format_ins['input'] = item['input']
            data_format_ins['actual_output'] = item['actual_output']
            data_format_ins['expected_output'] = item['expected_output']
            tag = False
            for x in save['data']:
                if is_same_eval_item(item,x):
                    for metric_data in x['cached_metrics_data']:
                        if metric_data['metric_metadata']['metric'] == metric.__name__:
                            if metric_data['metric_configuration']['threshold'] == metric.threshold and metric_data['metric_configuration']['evaluation_model'] == metric.evaluation_model and metric_data['metric_configuration']['strict_mode'] == metric.strict_mode and metric_data['metric_configuration']['include_reason'] == metric.include_reason:
                                tag = True
                                print("HAVE:")
                                print(x)
                                break
            if tag:
                continue
            test_case = LLMTestCase(
                input= item['input'],
                actual_output=item['actual_output'],
                context=[item['expected_output']],
            )
            try:
                metric.measure(test_case)
            except Exception as e:
                print(e)
                path.append(e)
                errors_format_ins['id'] = item['id']
                errors_format_ins['AnswerModel'] = item['AnswerModel']
                errors_format_ins['input'] = item['input']
                errors_format_ins['actual_output'] = item['actual_output']
                errors_format_ins['expected_output'] = item['expected_output']
                errors_format_ins['cached_metrics_data'][0]['metric_metadata']['metric'] = metric.__name__
                errors_format_ins['cached_metrics_data'][0]['metric_configuration'] = {'threshold':metric.threshold,'evaluation_model':metric.evaluation_model,'strict_mode':metric.strict_mode,'include_reason':metric.include_reason}
                error['data'].append(errors_format_ins.copy())
                print(errors_format_ins)
                with open(error_file,'w') as f:
                    json.dump(error,f,indent=4)
                continue
            metrics_format_ins['metric_metadata']['metric'] = metric.__name__
            metrics_format_ins['metric_metadata']['score'] = metric.score
            metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
            metrics_format_ins['metric_metadata']['reason'] = metric.reason
            
            metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
            metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
            metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

            metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
            metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
            metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
            metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason

            data_format_ins['cached_metrics_data'].append(metrics_format_ins.copy())
            flag = True
            for each in save['data']:
                if is_same_eval_item(each, data_format_ins):
                    each['cached_metrics_data'].append(metrics_format_ins.copy())
                    flag = False
                    break
            if flag:
                save['data'].append(data_format_ins.copy())
                print(data_format_ins)
            with open(save_file,'w') as f:
                json.dump(save,f,indent=4)

In [ ]:
def joinEvalErrorToData(errorFile:Union[str,Path],saveFile:Union[str,Path])->None:
    """_summary_
        the function is used to rerun the error item in the `errorFile` and append the results into the `saveFile` 
        
    Args:
        `errorFile` (Union[str,Path]): the JSON file saves the error item in the before running
        `saveFile` (Union[str,Path]): the JSON file saves the pass result
        
    Returns:
        None: the result will override the original file 
    """
    def is_same_eval_item(x,item):
        if x['id'] == item['id'] and x['AnswerModel'] == item['AnswerModel'] :
            return True
        return False
    with open(errorFile,'r') as f:
        data_er = json.load(f)
    with open(saveFile,'r') as f:
        data_sv = json.load(f)
    if data_er['fileName'] != data_sv['fileName']:
        print("The save_file does not match the error_file!")
        return
    error = {'fileName' :data_er['fileName'],'class':data_er['class'],'data':[]}
    while data_er['data']:
        item = data_er['data'].pop()
        data_format_ins = {
            'id':0,
            'AnswerModel':'',
            'input':'',
            'actual_output':'',
            'expected_output':None,
            'context':None,
            'retrieval_context':None,
            'cached_metrics_data':[]
        }
        metrics_format_ins = {
            'metric_metadata':{
                'metric':None,
                'success':True,
                'score':0.8,
                'reason':'',
                'statements':'',
                'verdicts':'',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        }
        errors_format_ins = {
            'id':0,
            'AnswerModel':'',
            'input':'',
            'actual_output':'',
            'expected_output':None,
            'retrieval_context':None,
            'cached_metrics_data':[
                {            
                    'metric_metadata':{
                        'metric':None,
                    },
                    'metric_configuration': {
                        'threshold': 0.5,
                        'evaluation_model': 'CustomLLM',
                        'strict_mode': False,
                        'include_reason': True
                    }
                }
            ]
        }
        data_format_ins['id'] = item['id']
        data_format_ins['AnswerModel'] = item['AnswerModel']
        data_format_ins['input'] = item['input']
        data_format_ins['actual_output'] = item['actual_output']
        data_format_ins['expected_output'] = item['expected_output']
        
        tag = False
        for x in data_sv['data']:
            if is_same_eval_item(item,x):
                for metric_data in x['cached_metrics_data']:
                    if metric_data['metric_metadata']['metric'] == item['cached_metrics_data'][0]['metric_metadata']['metric']:
                        if metric_data['metric_configuration']['threshold'] == item['cached_metrics_data'][0]['metric_configuration']['threshold'] and metric_data['metric_configuration']['evaluation_model'] == item['cached_metrics_data'][0]['metric_configuration']['evaluation_model'] and metric_data['metric_configuration']['strict_mode'] == item['cached_metrics_data'][0]['metric_configuration']['strict_mode'] and metric_data['metric_configuration']['include_reason'] == item['cached_metrics_data'][0]['metric_configuration']['include_reason']:
                            tag = True
                            print("HAVE:")
                            print(x)
                            break
        if tag:
            continue
        test_case = LLMTestCase(
            input= item['input'],
            actual_output=item['actual_output'],
            context=[item['expected_output']],
        )
        if item['cached_metrics_data'][0]['metric_metadata']['metric'] == 'Answer Relevancy':
            metric = AnswerRelevancyMetric(
                threshold=item['cached_metrics_data'][0]['metric_configuration']['threshold'],
                model = evaluateModel,
                include_reason=item['cached_metrics_data'][0]['metric_configuration']['include_reason']
            )
        elif item['cached_metrics_data'][0]['metric_metadata']['metric'] == 'Hallucination':
            metric = HallucinationMetric(
                threshold=item['cached_metrics_data'][0]['metric_configuration']['threshold'],
                model = evaluateModel,
                include_reason=item['cached_metrics_data'][0]['metric_configuration']['include_reason']
            )
        else:
            print("unkonwn metric!")
            continue
        
        try:
            metric.measure(test_case)
        except Exception as e:
            print(traceback.print_exc())
            errors_format_ins['id'] = item['id']
            errors_format_ins['AnswerModel'] = item['AnswerModel']
            errors_format_ins['input'] = item['input']
            errors_format_ins['actual_output'] = item['actual_output']
            errors_format_ins['expected_output'] = item['expected_output']
            errors_format_ins['cached_metrics_data'][0]['metric_metadata']['metric'] = metric.__name__
            errors_format_ins['cached_metrics_data'][0]['metric_configuration'] = {'threshold':metric.threshold,'evaluation_model':metric.evaluation_model,'strict_mode':metric.strict_mode,'include_reason':metric.include_reason}
            error['data'].append(errors_format_ins.copy())
            print(errors_format_ins)
            with open(errorFile,'w') as f:
                json.dump(data_er,f,indent=4)
            continue
        metrics_format_ins['metric_metadata']['metric'] = metric.__name__
        metrics_format_ins['metric_metadata']['score'] = metric.score
        metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
        metrics_format_ins['metric_metadata']['reason'] = metric.reason
        
        metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
        metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
        metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

        metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
        metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
        metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
        metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason

        data_format_ins['cached_metrics_data'].append(metrics_format_ins.copy())
        flag = True
        for each in data_sv['data']:
            if is_same_eval_item(each, data_format_ins):
                each['cached_metrics_data'].append(metrics_format_ins.copy())
                flag = False
                break
        if flag:
            data_sv['data'].append(data_format_ins.copy())
            print(data_format_ins)
        with open(saveFile,'w') as f:
            json.dump(data_sv,f,indent=4)
    
    with open(errorFile,'w') as f:
        json.dump(error,f,indent=4)

In [ ]:
class UnionFind:
    def __init__(self,n):
        self.n = n
        self.parent = [i for i in range(n)]
        self.size = n
        self.keyset = [1]*n
    def find(self,x):
        if self.parent[x]!= x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self,x,y):
        x = self.find(x)
        y = self.find(y)
        if x == y:
            return False
        if self.keyset[x] < self.keyset[y]:
            x,y = y,x
        self.parent[y] = x
        self.keyset[x] += self.keyset[y]
        self.size -= 1
        return True
    
    def is_connected(self,x,y):
        return self.find(x) == self.find(y)
    def get_size(self,x):
        return self.keyset[self.find(x)]
    def get_size_all(self):
        return self.size
def merge_same_item(file_path:Union[str,Path])->None:
    def is_same_eval_item(x,item):
        if x['id'] == item['id'] and  x['AnswerModel'] == item['AnswerModel'] :
            return True
        return False
    with open(file_path,'r') as f:
        data = json.load(f)
    check = []
    for item in data['data']:
        if len(item["cached_metrics_data"]) <2:
            check.append(item)
    check.sort(key=lambda x:(x['id'],x['AnswerModel']))
    uf = UnionFind(len(check))
    for i in range(len(check)):
        for j in range(i+1,len(check)):
            if is_same_eval_item(check[i],check[j]):
                uf.union(i,j)
    key_set = defaultdict(list)
    for i in range(len(check)):
        p = uf.find(i)
        key_set[p].append(check[i])
        
    mergeList =  list(key_set.values())
    for item in mergeList[:]:
        if len(item)<2:
            mergeList.remove(item)
    for x,y in mergeList:
        mergeItem = x.copy()
        mergeItem['cached_metrics_data'].append(y['cached_metrics_data'][0])
        data['data'].remove(x)
        data['data'].remove(y)
        data['data'].append(mergeItem)
    with open(file_path,'w') as f:
        json.dump(data,f,indent=4)
    print(file_path)

In [ ]:
# with open('data\\KoLA\\save\\3-4_kqapro_sample.json') as f:
#     data = json.load(f)
# for i,item in enumerate(data['data']):
#     item['id'] = "%04d"%i
# with open('data\\KoLA\\save\\3-4_kqapro_sample.json','w') as f:
#     json.dump(data,f,indent=4)

In [ ]:
# def is_same_eval_item(x,item):
#     if   x['AnswerModel'] == item['AnswerModel'] and x["input"] == item["input"] and x["expected_output"] == item["expected_output"]:
#         return True
#     return False
# with open('data\\KoLA\\save\\3-4_kqapro_sample.json') as f:
#     data_0 = json.load(f)
# exFile ='KoLA\\data\\3-4_kqapro_sample.json' #'data/KoLA/eval\\error\\3-4_kqapro_sample.json','data/KoLA/eval\\save\\3-4_kqapro_sample.json'
# with open(exFile) as f:
#     data = json.load(f)
# for item in data['data']:
#     for std in data_0['data']:
#         if is_same_eval_item(item,std):
#             item['id'] = std['id']
#             break
# with open(exFile,'w') as f:
#     json.dump(data, f, indent=4)

In [ ]:
# for dir,subdir,files in os.walk('./data/KoLA/eval/save'):
#     for file in files:
#         merge_same_item('./data/KoLA/eval/save/'+file)

In [ ]:
with open('./eval/error/2-5_DocRED_sample.json') as f:
    tmp = json.load(f)
tmp

In [ ]:
# with open('./eval/error/2-4_FewNERD++supervised_sample.json','r') as f:
#     data = json.load(f)
# for item in data['data']:
#     item['actual_output'] = item['actual_output'].replace("[/INST]",'')
#     item['actual_output'] = item['actual_output'].replace("[INST]",'')
# with open('./eval/error/2-4_FewNERD++supervised_sample.json','w') as f:
#     json.dump(data,f,indent=4)

In [ ]:
files_error = ['1-1_2_high_freq_ent_sample.json','1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json',  '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-4_kqapro_sample.json',  '3-6_r_KoRC++ood_sample.json',  '4-1_with_triples_sample.json', '4-2_r_with_triples_sample.json']
for file in files_error[::-1]:
    joinEvalErrorToData('./eval/error/'+file,'./eval/save/'+file)

In [ ]:

joinEvalErrorToData('./eval/error/2-5_DocRED_sample.json','./eval/save/2-5_DocRED_sample.json')

In [ ]:
path[0].text

In [ ]:
regex = re.compile("\"reason\":(.*?)\}",re.DOTALL)
matchStr = regex.findall(path[1])
# matchStr = regex.sub(lambda x:"\"reason\":\""+x.group(0).strip().replace("\"reason\":","").strip()[1:-1].replace('"','\\\"'),path[1])
matchStr

In [ ]:
matchStr[2].strip()[1:-1].replace('"','\\\"')

In [ ]:
regex.sub()

In [ ]:
path[1]

In [ ]:
['{"verdicts": [\n        {\n            "verdict": "idk"\n        },\n        {\n            "verdict": "no",\n            "reason": "The statement \'I am unable to provide an answer to your question\' acknowledges the inability to answer but doesn\'t provide any relevant information about the similarity with Rose."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'it is not clear what "Rose" refers to\' is relevant as it points out the need for clarification to determine similarity."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'Could you please provide more context or information?\' is relevant as it directly asks for additional details to establish the similarity with Rose."\n        }\n    ]}',
 '{"verdicts": [\n        {\n            "verdict": "no",\n            "reason": "The statement \'I\'m sorry,\' does not provide any information or similarity comparison with Rose."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'I need more context to answer your question.\' is relevant as it acknowledges the need for additional information to determine similarity with Rose."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'Could you please clarify what "Rose" refers to?\' is relevant as it seeks to understand the specific context or entity named Rose for a proper comparison."\n        },\n        {\n            "verdict": "yes",\n            "reason": "The statement \'What you are looking for in terms of a conceptually similar entity?\' is relevant as it asks for the criteria or aspects to consider when identifying a conceptually similar entity to Rose."\n        }\n    ]}']

In [ ]:
def has_answer_kilt(answers, text) -> bool:
    text = normalize_kilt(text)
    for single_answer in answers:
        single_answer = normalize_kilt(single_answer)
        if single_answer in text:
            return True
    return False


# answer normalization
def normalize_kilt(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
# for file in fileNames:
#     evaluate_file('./data/'+file,'./eval/save/'+file,'./eval/error/'+file)

In [ ]:
with open('./eval/save/2-1_COPEN++csj_sample.json') as f:
    data_view = json.load(f)
noEvalItem = []
for item in data_view['data']:
    if len(item['cached_metrics_data']) <2:
        noEvalItem.append(item)
noEvalItem

In [ ]:
os.listdir('./data/KoLA/save/')

In [ ]:
item = noEvalItem[0]

test_case = LLMTestCase(
    input= item['input'],
    actual_output=item['actual_output'],
    context=[item['expected_output']],
)
metric = AnswerRelevancyMetric(
    threshold=0.5,
    model = evaluateModel,
    include_reason=True,
)
metric.measure(test_case)


In [ ]:
for item in [noEvalItem[0]]:
    metrics_format_ins = {
        'metric_metadata':{
            'metric':None,
            'success':True,
            'score':0.8,
            'reason':'',
            'statements':'',
            'verdicts':'',
            'evaluationCost': 0
        },
        'metric_configuration': {
            'threshold': 0.5,
            'evaluation_model': 'CustomLLM',
            'strict_mode': False,
            'include_reason': True
        }
    }
    test_case = LLMTestCase(
        input= item['input'],
        actual_output=item['actual_output'],
        context=[item['expected_output']],
    )
    metric = AnswerRelevancyMetric(
        threshold=0.5,
        model = evaluateModel,
        include_reason=True,async_mode=False
    )
    metric.measure(test_case)
    
    metrics_format_ins['metric_metadata']['metric'] = metric.__name__
    metrics_format_ins['metric_metadata']['score'] = metric.score
    metrics_format_ins['metric_metadata']['success'] = metric.is_successful()
    metrics_format_ins['metric_metadata']['reason'] = metric.reason
    
    metrics_format_ins['metric_metadata']['statements'] = getattr(metric,'statements','')
    metrics_format_ins['metric_metadata']['verdicts'] = str(getattr(metric,'verdicts',''))
    metrics_format_ins['metric_metadata']['evaluationCost'] = metric.evaluation_cost

    metrics_format_ins['metric_configuration']['threshold'] = metric.threshold
    metrics_format_ins['metric_configuration']['strict_mode'] = metric.strict_mode
    metrics_format_ins['metric_configuration']['evaluation_model'] = metric.evaluation_model
    metrics_format_ins['metric_configuration']['include_reason'] = metric.include_reason
    
    print(metrics_format_ins)

In [23]:
data_eval = {"id":defaultdict(list),"Answer Relevancy":defaultdict(list),"Hallucination":defaultdict(list),}
for dir,subdir,files in os.walk('eval/save'):
    for file in files:
        with open(os.path.join(dir,file)) as f:
            data = json.load(f)
        for item in data['data']:
            for metric in item["cached_metrics_data"]:
                data_eval[metric["metric_metadata"]["metric"]][item['AnswerModel']].append(metric["metric_metadata"]["score"])
                # data_eval["id"][item['AnswerModel']].append(item["id"])


In [24]:
pd.DataFrame(data_eval)

,Answer Relevancy,Hallucination
baichuan2-13b-base,"[0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.5, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, ..."
baichuan2-13b-chat,"[1.0, 1.0, 0.75, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[1.0, 1.0, 0.0, 1.0, 0.3333333333333333, 0.0, ..."
baichuan2-7b-chat,"[1.0, 0.625, 1.0, 0.3333333333333333, 0.4, 1.0...","[1.0, 1.0, 0.0, 1.0, 0.6, 0.0, 1.0, 0.0, 1.0, ..."
infini-megrez-7b,"[1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 1.0, 0.6666666666666666, 1.0, ..."
llama-2-13b-chat,"[1.0, 1.0, 1.0, 0.4, 1.0, 0.5, 1.0, 0.33333333...","[1.0, 1.0, 0.0, 1.0, 0.6666666666666666, 0.0, ..."
llama-2-70b,"[0.24, 0.02127659574468085, 0.3333333333333333...","[1.0, 0.0, 0.6666666666666666, 1.0, 0.0, 1.0, ..."
llama-2-70b-chat,"[1.0, 1.0, 1.0, 0.8, 1.0, 1.0, 1.0, 0.8, 1.0, ...","[1.0, 0.0, 0.0, 1.0, 0.75, 0.0, 0.0, 1.0, 0.0,..."
llama-2-7b-chat,"[0.3333333333333333, 1.0, 1.0, 0.9, 0.5, 1.0, ...","[1.0, 0.0, 1.0, 0.3333333333333333, 1.0, 0.0, ..."
llama-3-8b-instruct,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 0.6666666666666666, 0.0, ..."
qwen-14b-chat,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 0.3333333333333333, 1.0, ..."


In [27]:
from collections import defaultdict
import pandas as pd
import numpy as np

In [24]:
data_id = defaultdict(dict)
for dir,subdir,files in os.walk('data/KoLA/eval/save'):
    for file in files:
        with open(os.path.join(dir,file)) as f:
            data = json.load(f)
        for item in data['data']:
            for metric in item["cached_metrics_data"]:
                if item['idx'] not in data_id[item['AnswerModel']]:
                    data_id[item['AnswerModel']][item["idx"]] = {}
                data_id[item['AnswerModel']][item["idx"]][metric["metric_metadata"]["metric"]] = metric["metric_metadata"]["score"]
        


In [25]:
pd.DataFrame(data_id).T

,1-1_2_high_freq_ent==00,1-1_2_high_freq_ent==01,1-1_2_high_freq_ent==02,1-1_2_high_freq_ent==03,1-1_2_high_freq_ent==04,1-2_1_low_freq_ent==00,1-2_1_low_freq_ent==04,1-2_1_low_freq_ent==02,1-2_1_low_freq_ent==01,1-2_1_low_freq_ent==03,...,4-2_r_without_triples==00,4-2_r_without_triples==01,4-2_r_without_triples==02,4-2_r_without_triples==03,4-2_r_without_triples==04,4-2_r_with_triples==00,4-2_r_with_triples==01,4-2_r_with_triples==02,4-2_r_with_triples==03,4-2_r_with_triples==04
baichuan2-13b-base,"{'Answer Relevancy': 0.5, 'Hallucination': 0.0}","{'Answer Relevancy': 0.5, 'Hallucination': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.5, 'Hallucination': 1.0}","{'Answer Relevancy': 0.5, 'Hallucination': 1.0}","{'Answer Relevancy': 0.5, 'Hallucination': 0.0}","{'Answer Relevancy': 0.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.0, 'Hallucination': 1.0}","{'Answer Relevancy': 0.5, 'Hallucination': 1.0}",{'Hallucination': 1.0},...,"{'Answer Relevancy': 1.0, 'Hallucination': 0.5}","{'Answer Relevancy': 0.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.4166666666666667, 'Hall...","{'Answer Relevancy': 0.0, 'Hallucination': 1.0}","{'Answer Relevancy': 1, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 0.0, 'Hallucination': 1.0}","{'Answer Relevancy': 1, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}"
baichuan2-13b-chat,"{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 0.75, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.3...","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}",...,"{'Answer Relevancy': 1.0, 'Hallucination': 0.1...","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 0.8888888888888888, 'Hall...",{'Hallucination': 0.0},"{'Answer Relevancy': 0.8333333333333334, 'Hall...","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}"
baichuan2-7b-chat,"{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 0.625, 'Hallucination': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.3333333333333333, 'Hall...","{'Answer Relevancy': 0.4, 'Hallucination': 0.6}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 0.0, 'Hallucination': 1.0}",...,"{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.4}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.9285714285714286, 'Hall...","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.8947368421052632, 'Hall...","{'Answer Relevancy': 1.0, 'Hallucination': 0.3...",{'Hallucination': 0.0},"{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}"
infini-megrez-7b,"{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 0.5, 'Hallucination': 0.0}","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.6...","{'Answer Relevancy': 1.0, 'Hallucination': 1.0}","{'Answer Relevancy': 0.0, 'Hallucination': 1.0}","{'Hallucination': 1.0, 'Answer Relevancy': 1.0}","{'Answer Relevancy': 1.0, 'Hallucination': 0.0}","{'Answer Relevancy': 0.0, 'Hallucination'

In [ ]:
data_id_mean = {"id":defaultdict(list),"Answer Relevancy":defaultdict(list),"Hallucination":defaultdict(list),}

In [20]:
pd.DataFrame(dict(data_id)['baichuan2-13b-base'])

,Answer Relevancy,Hallucination
2_high_freq_ent8186,0.5,0.0
1_low_freq_ent4818,0.5,1.0
1_low_freq_ent6889,1.0,0.0
2_high_freq_ent665,0.5,1.0
2_high_freq_ent4541,0.5,1.0
...,...,...
with_triples_sample_4,1.0,1.0
1_low_freq_ent3312,NaN,1.0
1103,NaN,1.0
114,NaN,1.0


In [4]:
data_eval_mean = {"Answer Relevancy":defaultdict(dict),"Hallucination":defaultdict(dict),}
for metric,modelEvalItem in data_eval.items():
    for model,evals in modelEvalItem.items():
        data_eval_mean[metric][model]["mean"] = np.mean(evals)
        data_eval_mean[metric][model]["variance"] = np.var(evals)

In [5]:
pd.DataFrame(data_eval_mean)

,Answer Relevancy,Hallucination
baichuan2-13b-base,"{'mean': 0.6114362741716677, 'variance': 0.166...","{'mean': 0.6769911504424779, 'variance': 0.216..."
baichuan2-13b-chat,"{'mean': 0.9458282458282459, 'variance': 0.015...","{'mean': 0.5367256637168142, 'variance': 0.232..."
baichuan2-7b-chat,"{'mean': 0.8908861703514317, 'variance': 0.066...","{'mean': 0.5594322344322344, 'variance': 0.224..."
infini-megrez-7b,"{'mean': 0.8334527612305389, 'variance': 0.090...","{'mean': 0.6245926394156482, 'variance': 0.222..."
llama-2-13b-chat,"{'mean': 0.886645419945961, 'variance': 0.0529...","{'mean': 0.6044247787610619, 'variance': 0.222..."
llama-2-70b,"{'mean': 0.6454967130863714, 'variance': 0.169...","{'mean': 0.5965442159934385, 'variance': 0.229..."
llama-2-70b-chat,"{'mean': 0.9011196789321788, 'variance': 0.046...","{'mean': 0.5789506953223766, 'variance': 0.232..."
llama-2-7b-chat,"{'mean': 0.8812358470896451, 'variance': 0.063...","{'mean': 0.6061688311688311, 'variance': 0.225..."
llama-3-8b-instruct,"{'mean': 0.9317500738553369, 'variance': 0.031...","{'mean': 0.5209860935524652, 'variance': 0.241..."
qwen-14b-chat,"{'mean': 0.9148030045351474, 'variance': 0.060...","{'mean': 0.528866413822166, 'variance': 0.2361..."


### MATH PROCESS AND EVAL

In [ ]:
#MATH
def get_LLM_Reply_MATH(filepath,savePath,errorPath):
    with open(filepath) as f:
        data = json.load(f)
    questionList = data['data']
    instructions = 'Only return the correct answer of the question.\n'
    for model in answerModelList:
        error = {'fileName':model.title()+'.json','model':model,'data':[]}
        save = {'fileName':model.title()+'.json','model':model,'data':[]}
        for index,item in enumerate(questionList):
            data_format_ins = {
                'id':0,
                'AnswerModel':'',
                'input':'',
                'actual_output':'',
                'expected_output':None,
                'is_correct':-1,
                'time':-1
            }
            if 'id' in item:
                data_format_ins['id'] = item['id']
            else:
                data_format_ins['id'] = model+'-%04d'% index
            prompt = instructions+item['input']
            data_format_ins['input'] = prompt
            print(prompt)
            data_format_ins['expected_output'] = item['expected_output']
            data_format_ins['AnswerModel'] = model
            print(model)
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model,top_p=0)
            end = time.perf_counter_ns()
            delta = end-start
            idx = 0
            while actual_output == "Cannot connect to the model "+model and idx<2:
                start = time.perf_counter_ns()
                actual_output =  LLMCompletions(prompt,modelName=model,top_p=0)
                end = time.perf_counter_ns()
                delta = end-start
                idx += 1
            if actual_output == "Cannot connect to the model "+model:
                error['data'].append({'id':data_format_ins['id'],"AnswerModel":model,"input":prompt,"expected_output":data_format_ins['expected_output']})
                continue
            print(idx,delta//1000_000,actual_output,sep='\t')
            data_format_ins['actual_output'] = actual_output
            data_format_ins['time'] = delta
            save['data'].append(data_format_ins.copy())
            print('*'*70)
        errorItemFinal = []
        while error['data']:
            item = error['data'].pop()
            data_format_ins['id'] = item['id']
            model = item['AnswerModel']
            data_format_ins['AnswerModel'] = model
            prompt = item['input']
            data_format_ins['input'] = prompt
            data_format_ins['expected_output'] = item['expected_output']
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model,top_p=0)
            end = time.perf_counter_ns()
            delta = end-start
            idx = 0
            while actual_output == "Cannot connect to the model "+model and idx<2:
                print('\t'+str(idx)+'\ttest')
                start = time.perf_counter_ns()
                actual_output =  LLMCompletions(prompt,modelName=model,top_p=0)
                end = time.perf_counter_ns()
                delta = end-start
                idx += 1
            if actual_output == "Cannot connect to the model "+model:
                errorItemFinal.append(item)
                print("[error]:\t"+str(errorItemFinal[-1]))
                continue
            print(idx,delta,actual_output,sep='\t')
            data_format_ins['actual_output'] = actual_output
            data_format_ins['time'] = delta
            save['data'].append(data_format_ins.copy())
        with open(os.path.join(savePath,model+'.json'),'w',encoding='utf-8') as out:
            json.dump(save,out,indent=4)
        if errorItemFinal:
            for i in errorItemFinal:
                error['data'].append(i)
            with open(os.path.join(errorPath,model+'.json'),'w',encoding='utf-8') as out:
                json.dump(error,out,indent=4)
        print('='*70)

In [ ]:
get_LLM_Reply_MATH('./data/math401/math50.json','./data/math401/save','./data/math401/error')

In [ ]:
# with open('./data/math401/math50.json') as f:
#     data = json.load(f)
# data_new  = {'fileName': data['fileName'],'data':[]}
# for item in data['data']:
#     data_format = {'id':item['id'],'input':item['input'],'expected_output':item['expected_output']}
#     data_new['data'].append(data_format.copy())
# with open('./data/math401/math50.json','w') as f:
#     json.dump(data_new,f,indent=4)

In [ ]:
with open('./data/math401/math50.json','r') as f:
    data_eval  = json.load(f)
data_eval['data']

In [ ]:
# with open('./data/math401/save/infini-megrez-7b.json','r') as f:
#     data_401 = json.load(f)
#     data_401_new = {'fileName': data_401['fileName'],'data':[]}
#     for item in data_401['data']:
#         for x in data['data']:
#             if item['input'] =='Only return the correct answer of the question.\n'+ x['input']:
#                 item['id'] = x['id']
#                 item['is_correct'] = -1
#                 data_401_new['data'].append(item.copy())
# with open('./data/math401/save/infini-megrez-7b.json','w') as f:
#     json.dump(data_401_new,f,indent=4)

In [ ]:
def sort_result(filePath:Union[str,Path])->None:
    """
    _summary_
        the function is used to sort the results saved in the JSON `filepath` by `id` field
        
    Args:
        `filePath` (Union[str,Path]): the JSON file path

    Returns:
        None: the result will override the original `filePath` 
    """
    with open(filePath,'r') as f:
        data = json.load(f)
    data['data'].sort(key = lambda x:x['id'])
    with open(filePath,'w') as f:
        json.dump(data,f,indent=4)


In [ ]:
def addField(file:Union[str,Path],field:str,default:Any)->None:
    """
    _summary_
        the function is used to add a field into the `.json` `file`

    Args:
        `file` (Union[str,Path]): the filename ,the file should be JSON file
        `field` (str): the field name
        `default` (Any): the default value of the added field,the value will add or subtract a Random from -500_000_000 to 500_000_000 if the field is `time`
        
    Returns:
        None: the result will override the original `file` 
    """
    with open(file,'r') as f:
        data = json.load(f)
    for item in data['data']:
        if field not in item:
            item[field] = default
            if field == 'time':
                item[field] += random.randint(-500_000_000,500_000_000)
    with open(file,'w') as f:
        json.dump(data,f,indent = 4)

In [ ]:
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        addField(os.path.join(dir,file),'extract_answer',None)

In [ ]:
def getNumberAnswer(text:str)->Union[float,None]:
    """_summary_
        the function is used to extract the number from the text

    Args:
        `text` (str): the text contains the number
        
    Returns:
        Union[float,None]: the number in the `text`, or None if the `text` does not contain the number
    """
    text = text.split('=')
    if text:
        text = text[-1]
        regex = re.compile('([+-]?\d+[,0-9]*[.]?[0-9]*)')
        ret = regex.findall(text)
        if ret:
            return eval(ret[-1].replace(',',''))
        else:
            return None

In [ ]:
def processAnswer(file:Union[str,Path]):
    """
    _summary_:
        the function is used to process the answer that the LLM returned , namely extracting the number from the `actual_output` and save as `extract_answer`, and transform `expected_output` from str to number[int,float] 

    Args:
        `file` (Union[str,Path]): the filename of the file saving the answer created by the LLM
        
    Returns:
        None : the result will be written into the original `file`
    """
    with open(file,'r') as f:
        data = json.load(f)
    for item in data['data']:
        if item['is_correct'] == -1:
            item['extract_answer'] = getNumberAnswer(item['actual_output'])
            item['expected_output'] = eval(item['expected_output'])
            if item['extract_answer'] is None :
                item['is_correct'] = 0
            else:
                item['is_correct'] =1 if abs(item['expected_output']-item['extract_answer']) < 1e-3 else 0
    with open(file,'w') as f:
        json.dump(data,f,indent = 4)

In [ ]:
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        processAnswer(os.path.join(dir,file))

In [ ]:
# for dir,subdir ,files in os.walk('./data/math401/save'):
#     for file in files:
#         with open(os.path.join(dir,file),'r') as f:
#             data = json.load(f)
#         for item in data['data']:

#             if item['is_correct']:
#                 item['is_correct'] = 1
#             else:
#                 item['is_correct'] = 0
#         with open(os.path.join(dir,file),'w') as f:
#             json.dump(data,f,indent = 4)

In [45]:

def calculate_accuracy(file:Union[str,Path],return_tuple:bool = False)->Union[float,tuple[str,float]]:
    """_summary_
        the function calculates the accuracy of the model answers on the `math50` dataset saved in `file`

    Args:
        `file` (Union[str,Path]): the path to the JSON `file` containing a model answer results
        `return_tuple` (bool, optional): if the value is True,the function will return a tuple (modelName,accuracy),else only return accuracy `[0,1]`. 
                    Defaults to False.

    Returns:
        Union[float,tuple[str,float]]: if return_tuple is True,return a tuple containing both modelName and accuracy in format `(modelName:str,accuracy:float)`,else only return the accuracy in [0,1]
    """
    with open(file,'r') as f:
        data = json.load(f)
    correct = 0
    for item in data['data']:
        correct += item['is_correct']
    accuracy =  correct/len(data['data'])
    if return_tuple:
        return (data['model'],accuracy)
    return accuracy

def calculate_nan_ratio(file:Union[str,Path],return_tuple:bool = False)->Union[float,tuple[str,float]]:
    """_summary_
        the function calculates the no number ratio of the LLM answers on the `math50` dataset saved in `file`

    Args:
        `file` (Union[str,Path]): the path to the JSON `file` containing a model answer results
        `return_tuple` (bool, optional): if the value is True,the function will return a tuple (modelName,nan_ratio),else only return nan_ratio `[0,1]`. Defaults to False.

    Returns:
        Union[float,tuple[str,float]]: if return_tuple is True,return a tuple containing both modelName and nan_ratio in format `(modelName:str,nan_ratio:float)`,else only return the nan_ratio in [0,1]
    """
    with open(file,'r') as f:
        data = json.load(f)
    nan = 0
    for item in data['data']:
        if item['extract_answer'] is None:
            nan += 1
    nan_ratio = nan/len(data['data'])
    if return_tuple:
        return (data['model'],nan_ratio)
    return nan_ratio

In [46]:
def RE(y_true:Union[int,float],y_pred:Union[int,float])->float:
    return min(10,abs(y_true-y_pred)/max(abs(y_true),1))
def calculate_RE(file:Union[str,Path],return_tuple:bool = False)->Union[List[float],tuple[str,List[float]]]:
    
    with open(file,'r') as f:
        data = json.load(f)
    RE_list = []
    for item in data['data']:
        ret = 0
        y_true = item['expected_output']
        y_pred = item['extract_answer']
        if y_pred is None:
            ret = 10
        else:
            ret = RE(y_true,y_pred)
        RE_list.append(ret)
    if return_tuple:
        return (data['model'],RE_list)
    return RE_list

In [ ]:
# data_list = {'eval':'math','eval_dataset':'math50.json','data':[]}
# for dir,subdir ,files in os.walk('./data/math401/save'):
#     for file in files:
#         tmp = calculate_accuracy(os.path.join(dir,file),return_tuple=True)
#         nan_ratio = calculate_nan_ratio(os.path.join(dir,file))
#         RE_List = calculate_RE(os.path.join(dir,file))
#         data_list['data'].append({'model':tmp[0],'Accuracy':tmp[1],'Nan_Ratio':nan_ratio,'RE_List':RE_List[:]})
# with open('./data/math401/math50_eval_result.json','w') as f:
#     json.dump(data_list,f,indent=4)

In [47]:
data = {}
for dir,subdir ,files in os.walk('./data/math401/save'):
    for file in files:
        tmp = calculate_accuracy(os.path.join(dir,file),return_tuple=True)
        nan_ratio = calculate_nan_ratio(os.path.join(dir,file))
        RE_List = calculate_RE(os.path.join(dir,file))
        data[tmp[0]] = {'Accuracy':tmp[1],'Nan_Ratio':nan_ratio,'RE_List':RE_List[:]}
df = pd.DataFrame(data).T.sort_index()
df.to_json('./data/math401/math50_eval_result_df.json',indent=4)

### codeEval

In [ ]:
from evalplus.data import get_human_eval_plus

In [ ]:
os.environ.get("HUMANEVAL_OVERRIDE_PATH", None)

In [ ]:
with open('./data/codeEval/data.json','r') as f:
    data_eval = json.load(f)


In [ ]:
# with open('./data/codeEval/data.jsonl','w') as fw:
#     for item in data.keys():
#         fw.write(json.dumps(data[item])+'\n')

In [ ]:
data_eval = []
with open('./data/codeEval/code_sanitize/qwen-72b.jsonl') as f:
    for line in f.readlines():
        data_eval.append(json.loads(line))


In [ ]:
data_eval[1]['solution']

In [ ]:
with open('./data/codeEval/data.json') as f:
    data_o = json.load(f)

In [ ]:
from evalplus.evaluate import main

In [ ]:
from sys import argv

In [ ]:
argv.pop()

In [ ]:
local_args = argv
argv

In [ ]:
local_args.append('--dataset=humaneval')
local_args.append('--samples=./data/codeEval/code_sanitize/qwen-72b.jsonl')
local_args.append('--test-details')

In [ ]:
with open('data/codeEval/data.json', 'r') as f:
    data_eval = json.load(f)

In [ ]:
import time
from func_timeout import *

In [ ]:
def execute(code:str,inputs:List,entry_point:str,record_time=True,timeout:float = 10)->List:
    env = {}
    n = len(inputs)
    local = {}
    try:
        exec(code,None,local)
        if len(local) != 1:
            exec(code,env)
        else:
            env = local
    except Exception as e:
        print(e)
        if record_time:
            return (["SyntaxError"]*n,[(1<<31)-1]*n)
        return ["SyntaxError"]*n
    if entry_point in env:
        fn = env[entry_point]
    else:
        if record_time:
            return (["NotImplemented"]*n,[(1<<31)-1]*n)
        return ["NotImplemented"]*n
    @func_set_timeout(timeout)
    def get(inp,record_time:bool=True):
        try:
            start = time.perf_counter_ns()
            ret = fn(*inp) #if fn.__code__.co_argcount > 1 else fn(inp)
            end = time.perf_counter_ns()
            delta = (end-start)//1000_000
        except Exception as e:
            print(e,fn)
            ret = "SyntaxError"
            delta = (1<<31)-1
        if isinstance(ret,NotImplementedError) or str(ret) == str(NotImplemented):
            ret = "NotImplemented"
        if record_time:
            return (ret,delta)
        return ret
        
    ret = []
    rtime = []
    for inp in inputs:
        try:
            res,delta = get(inp)
            ret.append(res)
            rtime.append(delta)
        except FunctionTimedOut as e:
            print(e)
            ret.append("TimeLimitExceeding")
            rtime.append((1<<31)-1)
    # try:
    #     exec(code,None,local)
    #     for key in local.keys:
    #         del env[key]
    # except:
    #     print("del error")
    del env
    if record_time:
        return ret,rtime
    return ret

In [ ]:
def trust_execute(code:str,inputs:List,entry_point:str,record_time=True)->List:
    env = {}
    exec(code,None, env)
    fn = env[entry_point]
    ret = []
    rtime = []
    for i,inp in enumerate(inputs):
        if record_time:
            start = time.perf_counter_ns()
            tmp = fn(*inp) #if fn.__code__.co_argcount > 1 else fn(inp)
            end = time.perf_counter_ns()
            ret.append(tmp)
            rtime.append((end-start)//1000_000)
        else:
            tmp = fn(*inp) #if fn.__code__.co_argcount > 1 else fn(inp)
            ret.append(tmp)
    if record_time:
        return ret,rtime
    return ret

In [ ]:
def standardExecute(dataset:Union[str,Path],inplace = False,is_force_override = False):
    def trust_execute(code:str,inputs:List,entry_point:str,record_time=True)->List:
        env = {}
        exec(code,None, env)
        fn = env[entry_point]
        ret = []
        rtime = []
        for i,inp in enumerate(inputs):
            if record_time:
                start = time.perf_counter_ns()
                ret.append(fn(*inp))
                end = time.perf_counter_ns()
                rtime.append((end-start)//1000_000)
            else:
                ret.append(fn(*inp))
        if record_time:
            return ret,rtime
        return ret
    if isinstance(dataset, str):
        dataset = Path(dataset)
    if dataset.suffix == ".json":
        with open(dataset,'r') as f:
            data_origin = json.load(f)
    elif dataset.suffix == '.jsonl':
        with open(dataset,'r') as f:
            data_origin = {}
            for line in f:
                item = json.loads(line)
                data_origin[item['task_id']] = item
    else:
        raise Exception(f"{dataset.suffix} is not supported")
    """            
        {"{{task_id}}":{
            "task_id":str,
            "prompt":str,
            "entry_point":str,
            "canonical_solution":str[code],
            "base_input":List[input],
            "plus_input":List[input],
            "base":List[output],
            "base_time":List[float],
            "plus":List[output],
            "plus_time":List[float],
            }
        }
    """
    save = dataset.stem + "_with_output.json"
    if os.path.exists(os.path.join(dataset.parent, save)) and not is_force_override:
        print("The output file already exists")
        return 
    for task_id,problem in data_origin.items():
        oracle = data_origin[task_id]
        oracle["base"], oracle["base_time"] = trust_execute(
            problem["prompt"] + problem["canonical_solution"],
            problem["base_input"],
            problem["entry_point"],
            record_time=True,
        )
        oracle["plus"], oracle["plus_time"] = trust_execute(
            problem["prompt"] + problem["canonical_solution"],
            problem["plus_input"],
            problem["entry_point"],
            record_time=True,
        )
    if not inplace:
        with open(os.path.join(dataset.parent,save),'w') as f:
            json.dump(data_origin,f,indent=4)
    else:
        with open(dataset,'w') as f:
            json.dump(data_origin,f,indent=4)

In [ ]:
def codeExecute(file_path:Union[str,Path],dataset:Union[str,Path],exec_save_path:Union[str,Path] = None,is_force_override = False)->None:
    if isinstance(dataset, str):
        dataset = Path(dataset)
    if dataset.suffix == ".json":
        with open(dataset,'r') as f:
            data_origin = json.load(f)
    elif dataset.suffix == '.jsonl':
        with open(dataset,'r') as f:
            """            
                {"{{task_id}}":{
                    "task_id":str,
                    "prompt":str,
                    "entry_point":str,
                    "canonical_solution":str[code],
                    "base_input":List[input],
                    "plus_input":List[input],
                    "base":List[output],
                    "base_time":List[float],
                    "plus":List[output],
                    "plus_time":List[float],
                    }
                }
            """
            data_origin = {}
            for line in f:
                item = json.loads(line)
                data_origin[item['task_id']] = item
    else:
        raise Exception(f"{dataset.suffix} is not supported")
    
    if isinstance(file_path, str):
        file_path = Path(file_path)
    if file_path.suffix == ".json":
        with open(file_path,'r') as f:
            data_eval = json.load(f)
    elif file_path.suffix == '.jsonl':
        with open(file_path,'r') as f:
            # {"task_id":"solution",}
            data_eval = {}
            for line in f:
                item = json.loads(line)
                task_id = item['task_id']
                data_eval[task_id] = item['solution'] if 'solution' in item else data_origin[task_id]['prompt']+item['completion']
    else:
        raise Exception(f"{file_path.suffix} is not supported")
    """
        {
            "{{task_id}}":{
                "info":{
                    "task_id":"{{task_id}}",
                    "prompt":"{{prompt}}",
                    "entry_point":"{{entry_point}}",
                    "canonical_solution":"{{canonical_solution}}",
                    "base_input":"{{base_input}}",
                    "plus_input":"{{plus_input}}",
                    "atol":float,
                },
                "expected_output":{
                    "base":"{{base}}",
                    "base_time":"{{base_time}}",
                    "plus":"{{plus}}",
                    "plus_time":"{{plus_time}}",
                },
                "code_LLM":"{{solution}}",
                "actual_output":{
                    "base":"{{base}}",
                    "base_time":"{{base_time}}",
                    "plus":"{{plus}}",
                    "plus_time":"{{plus_time}}",
                }
            }
        }
    """
    output = {}
    save = file_path.stem + "_execute_results.json"
    if not exec_save_path:
        parent = file_path.parent
    else:
        if isinstance(exec_save_path, str):
            exec_save_path = Path(exec_save_path)
        parent = exec_save_path if os.path.isdir(exec_save_path) else exec_save_path.parent
    if os.path.exists(os.path.join(parent,save)) and not is_force_override:
        print("The executeResults file already exists")
        return
    for task_id,problem in data_origin.items():
        oracle = {}
        oracle["base"], oracle["base_time"] = problem['base'],problem['base_time']
        oracle["plus"], oracle["plus_time"] = problem['plus'],problem['plus_time']
        if task_id not in output:
            output[task_id] = {}
        output[task_id]['info'] = problem
        output[task_id]["expected_output"] = oracle
    for task_id,solution in data_eval.items():
        oracle = {}
        oracle["base"], oracle["base_time"] = execute(
            solution,
            data_origin[task_id]["base_input"],
            data_origin[task_id]["entry_point"],
            record_time=True,
        )
        oracle["plus"], oracle["plus_time"] = execute(
            solution,
            data_origin[task_id]["plus_input"],
            data_origin[task_id]["entry_point"],
            record_time=True,
        )
        if task_id not in output:
            output[task_id] = {}
        output[task_id]['code_LLM'] = solution
        output[task_id]["actual_output"] = oracle
    with open(os.path.join(parent,save), "w") as f:
        json.dump(output, f, indent=4,)
    print(save)
def is_floats(x) -> bool:
    # check if it is float; List[float]; Tuple[float]
    if isinstance(x, float):
        return True
    if isinstance(x, (list, tuple)):
        return all(isinstance(i, float) for i in x)
    if isinstance(x, np.ndarray):
        return x.dtype == np.float64 or x.dtype == np.float32
    return False

In [ ]:
def codeEvaluate(exec_file:Union[str,Path],eval_save_path:Union[str,Path] = None,is_force_override:bool = False):
    def equal(exp:List,actual:List,atol = 0)->List[bool]:
        ret_correct = []
        if is_floats(exp):
            if atol == 0:
                atol = 1e-6
            for i in range(len(exp)):
                if isinstance(actual[i],str) or not np.isclose(exp[i],actual[i],atol=atol):
                    ret_correct.append(False)
                else:
                    ret_correct.append(True)
        else:
            for i in range(len(exp)):
                if actual[i] in ["NotImplemented","TimeLimitExceeding","SyntaxError"] or actual[i] != exp[i]:
                    ret_correct.append(False)
                else:
                    ret_correct.append(True)
        return ret_correct
    if isinstance(exec_file, str):
        exec_file = Path(exec_file)
    if exec_file.suffix != ".json":
        raise Exception(f"{exec_file.suffix} is not supported")
    with open(exec_file,'r') as f:
        data = json.load(f)
    if not eval_save_path:
        parent = exec_file.parent
    else:
        if isinstance(eval_save_path,str):
            eval_save_path = Path(eval_save_path)
        parent = eval_save_path if os.path.isdir(eval_save_path) else eval_save_path.parent
            
    save = exec_file.stem.replace('_execute_results','') + "_eval_results.json"
    if os.path.exists(os.path.join(parent,save)) and not is_force_override:
        print("The evalResults file already exists")
        return 
    eval_result = {}
    """
        {
            "percision":{
                "base":float,
                "plus":float,
                "mean":float
            },
            "AC":int,
            "PASS":float,
            "syntax_error_ratio":{
                "base":float,
                "plus":float
                "mean":float
            },
            "accuracy_list":{
                "base_accuracy_list":List[float],
                "plus_accuracy_list":List[float]
            },
            "syntax_error_list":{
                "base_syntax_error_list":List[bool],
                "plus_syntax_error_list":List[bool]
            },
            "notImplemented_ratio_list":List[float],
            "{{task_id}}":{
                "base_eval":List[bool],
                "base_num":int,
                "base_AC":int,
                "base_TLE":int,
                "base_accuracy":float,
                "base_exp_time":List[time],
                "base_actual_time":List[time],
                
                "plus_eval":List[bool],
                "plus_num":int,
                "plus_AC":int,
                "plus_TLE":int,
                "plus_accuracy":float,
                "plus_exp_time":List[time],
                "plus_actual_time":List[bool],
                
                "accuracy":float
            }
        }
    """
    eval_result['percision'] = {'base':0,'plus':0,"mean":0}
    eval_result['AC'] = 0
    eval_result['PASS'] = 0
    AC_num = 0
    eval_result['syntax_error_ratio'] = {'base':0,"plus":0,"mean":0}
    base_accuracy_list = []
    plus_accuracy_list = []
    accuracy_list = {"base_accuracy_list":base_accuracy_list,"plus_accuracy_list":plus_accuracy_list}
    eval_result['accuracy_list'] = accuracy_list
    base_syntax_error_list = []
    plus_syntax_error_list = []
    eval_result['syntax_error_list'] ={ "base_syntax_error_list":base_syntax_error_list,"plus_syntax_error_list":plus_syntax_error_list}
    notImplemented_list = []
    eval_result['notImplemented_ratio_list'] = notImplemented_list
    for task_id,eval in data.items():
        oracle = {}
        if 'atol' in eval['info']:
            atol = eval['info']['atol']
        else:
            atol = 0
        base_exp = eval['expected_output']['base']
        base_actual = eval['actual_output']['base']
        base_correct = equal(base_exp, base_actual,atol)
        
        oracle['base_num'] = len(base_correct)
        oracle['base_AC'] = sum(base_correct)
        oracle['base_TLE'] = base_actual.count("TimeLimitExceeding")
        oracle['base_SE'] = base_actual.count("SyntaxError")
        oracle['base_NoImplemented'] = base_actual.count("NotImplemented")
        oracle['base_accuracy'] = np.mean(base_correct)
        base_accuracy_list.append(oracle['base_accuracy'])
        base_syntax_error_list.append(oracle['base_SE']/oracle['base_num'])
        
        plus_exp = eval['expected_output']['plus']
        plus_actual = eval['actual_output']['plus']
        plus_correct = equal(plus_exp,plus_actual,atol)
        
        oracle['plus_num'] = len(plus_correct)
        oracle['plus_AC'] = sum(plus_correct)
        oracle['plus_TLE'] = plus_actual.count("TimeLimitExceeding")
        oracle['plus_SE'] = plus_actual.count("SyntaxError")
        oracle['plus_NoImplemented'] = plus_actual.count("NotImplemented")
        oracle['plus_accuracy'] = np.mean(plus_correct)
        plus_accuracy_list.append(oracle['plus_accuracy'])
        plus_syntax_error_list.append(oracle['plus_SE']/oracle['plus_num'])
        
        oracle['accuracy'] = np.mean(base_correct+plus_correct)
        if oracle['accuracy'] == 1:
            AC_num += 1
        
        oracle['base_exp_time'] = eval['expected_output']['base_time']
        oracle['base_actual_time'] = eval['actual_output']['base_time']
        oracle['plus_exp_time'] = eval['expected_output']['plus_time']
        oracle['plus_actual_time'] = eval['actual_output']['plus_time']
        
        notImplemented_list.append((oracle['base_NoImplemented']+oracle['plus_NoImplemented'])/(oracle['base_num']+oracle['plus_num']))
        
        eval_result[task_id] = oracle
    eval_result['percision']['base'] = np.mean(base_accuracy_list)
    eval_result['percision']['plus'] = np.mean(plus_accuracy_list)
    eval_result['percision']['mean'] = np.mean(base_accuracy_list+plus_accuracy_list)
    eval_result['syntax_error_ratio']['base'] = np.mean(base_syntax_error_list)
    eval_result['syntax_error_ratio']['plus'] = np.mean(plus_syntax_error_list)
    eval_result['syntax_error_ratio']['mean'] = np.mean(base_syntax_error_list+plus_syntax_error_list)
    eval_result['AC'] = AC_num
    eval_result['PASS'] = AC_num/len(data)
    with open(os.path.join(parent,save), "w") as f:
        json.dump(eval_result, f, indent=4,)
    print(save)

In [ ]:
dir = './data/codeEval/code_sanitize'
files = ['baichuan2-13b-base.jsonl', 'baichuan2-13b-chat.jsonl', 'baichuan2-7b-chat.jsonl', 'infini-megrez-7b.jsonl', 'llama-2-13b-chat.jsonl', 'llama-2-70b.jsonl', 'llama-2-7b-chat.jsonl', 'qwen-14b-chat.jsonl', 'qwen-72b-chat.jsonl', 'qwen-72b.jsonl', 'qwen-7b-chat.jsonl', 'qwen1.5-14b-chat.jsonl', 'qwen1.5-72b.jsonl', 'qwen1.5-7b-chat.jsonl']

In [ ]:
# for dir,subdir,files in os.walk('./data/codeEval/code_sanitize'):
#     for file in files:
#         codeExecute(os.path.join(dir,file),"./data/codeEval/data_with_output.json",exec_save_path = './data/codeEval/code_execute',is_force_override=True)
#         codeEvaluate(os.path.join('./data/codeEval/code_execute',file.replace(".jsonl","_execute_results.json")),eval_save_path = './data/codeEval/code_eval',is_force_override=True)

In [ ]:
with open('./data/codeEval/data_with_output.json') as f:
    data_o = json.load(f)

In [ ]:
data_o['HumanEval/6']['base']

In [ ]:
data = []
with open('./data/codeEval/code_sanitize/qwen-72b.jsonl', 'r') as f:
    for line in f.readlines():
        data.append(json.loads(line))
data[1]#['HumanEval/6']

In [ ]:
env = {}
exec(data[1]['solution'],None,env)

In [ ]:
data_o['HumanEval/6']['base_input']

In [ ]:
fn = env['parse_nested_parens']

In [ ]:
for inp in data_o['HumanEval/6']['base_input']:
    print(fn(*inp))

In [ ]:
env = {}
local = {}
code = "def calling():\n    return called()\ndef called():\n    return 'success'"
exec(code,env,local)
# fn = env['calling']

In [ ]:
files[7]

In [ ]:
standardExecute('./data/codeEval/data.json',is_force_override=True)

In [ ]:
for i in range(14):
# codeExecute(os.path.join(dir,files[i]),"./data/codeEval/data_with_output.json",exec_save_path = './data/codeEval/code_execute',is_force_override=True)
    codeEvaluate(os.path.join('./data/codeEval/code_execute',files[i].replace(".jsonl","_execute_results.json")),eval_save_path = './data/codeEval/code_eval',is_force_override=True)

In [ ]:
def equal(exp:List,actual:List,atol = 0)->List[bool]:
    ret_correct = []
    if is_floats(exp):
        if atol == 0:
            atol = 1e-6
        for i in range(len(exp)):
            if isinstance(actual[i],str) or not np.isclose(exp[i],actual[i],atol=atol):
                ret_correct.append(False)
            else:
                print("exp:\t",exp[i])
                print("actual:\t",actual[i])
                print('='*70)
                ret_correct.append(True)
    else:
        for i in range(len(exp)):
            if actual[i] in ["NotImplemented","TimeLimitExceeding","SyntaxError"] or actual[i] != exp[i]:
                ret_correct.append(False)
            else:
                print("exp:\t",exp[i])
                print("actual:\t",actual[i])
                print('='*70)
                ret_correct.append(True)
    return ret_correct

In [ ]:
with open('./data/codeEval/code_execute/qwen-72b_execute_results.json') as f:
    data = json.load(f)
for task_id ,problem in data.items():
    exp = data[task_id]['expected_output']['plus']
    actual = data[task_id]['actual_output']['plus']
    equal(exp,actual)

In [ ]:
data_eval = {'accuracy':{},"pass":{},"AC":{}}
for dir,subdir,files in os.walk('./data/codeEval/code_eval'):
    for file in files:
        model = file.replace('_eval_results.json','')
        with open(os.path.join(dir,file)) as f:
            data = json.load(f)
        data_eval['accuracy'][model] = data['percision']['mean']
        data_eval['pass'][model] = data['PASS']
        data_eval['AC'][model] = data['AC']

In [ ]:
pd.DataFrame(data_eval).sort_values(by="AC",ascending=False)

In [1]:
import json,os
from pathlib import Path

In [7]:
for dir,subdir,files in os.walk("E:\\Repository\\KoLA\\Sample_Data"):
    for file in files:
        if file.endswith(".json"):
            with open(os.path.join(dir,file)) as f:
                data = json.load(f)
            instructions = data["adapter_spec"]["instructions"]
            prefix = file.replace(".json","").replace("_sample","")
            for i,ins in enumerate(data["request_states"]):
                ins["instance"]["idx"] = prefix+"=="+("%02d"%i)
            with open(os.path.join(dir,file),'w') as f:
                json.dump(data,f,indent=4)

In [8]:
for dir,subdir,files in os.walk("E:\\Repository\\KoLA\\Sample_Data"):
    print(files)

['1-1_2_high_freq_ent_sample.json', '1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json', 'README.md']


In [18]:
dir_o = "E:\\Repository\\KoLA\\Sample_Data"
dir_data = "E:\\学习资料\\2023-2024第二学期_大二下\\信息存储与检索\\小组作业\\data\\KoLA\\save"
dir_eval = "E:\\学习资料\\2023-2024第二学期_大二下\\信息存储与检索\\小组作业\\eval\\save"
dir_eval_error = "E:\\学习资料\\2023-2024第二学期_大二下\\信息存储与检索\\小组作业\\eval\\error"
fileList = ['1-1_2_high_freq_ent_sample.json', '1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json']
for file in fileList:
    with open(os.path.join(dir_o,file)) as f:
        data_origin = json.load(f)
    # dir_data = dir_eval
    # dir_data = dir_eval_error
    if not os.path.exists(os.path.join(dir_data,file)):
        continue
    with open(os.path.join(dir_data,file)) as f:
        data_save = json.load(f)
    if data_save['fileName'] != file:
        print("Not Match "+file)
        continue
    instructions = data_origin["adapter_spec"]["instructions"]
    for ins in data_origin["request_states"]:
        inp = ins["instance"]["input"]["text"]
        idx = ins["instance"]["idx"]
        expected_output = ins["instance"]["references"][0]["output"]["text"]
        for item in data_save['data']:
            if item["expected_output"] == expected_output and item["input"] == instructions+"\n"+inp :
                item["idx"] = idx
    with open(os.path.join(dir_data,file),'w') as f:
        json.dump(data_save,f,indent=4)

In [2]:
from copy import deepcopy

In [3]:
data_view = {}
for dir,subdir,files in os.walk('data/KoLA/eval/save'):
    for file in files:
        data_id = defaultdict(dict)
        with open(os.path.join(dir,file)) as f:
            data = json.load(f)
        for item in data['data']:
            for metric in item["cached_metrics_data"]:
                if metric["metric_metadata"]["metric"] not in data_id[item['AnswerModel']]:
                    data_id[item['AnswerModel']][metric["metric_metadata"]["metric"]] = []
                data_id[item['AnswerModel']][metric["metric_metadata"]["metric"]].append(metric["metric_metadata"]["score"])
        for key,value in data_id.items():
            for k,v in value.items():
                data_id[key][k] = np.mean(v)
        data_view[file] = data_id.copy()

data_view_ar = deepcopy(data_view)
data_view_h = deepcopy(data_view)
for key,value in data_view_ar.items():
    for k,v in value.items():
        data_view_ar[key][k] = v['Answer Relevancy'] if 'Answer Relevancy' in v else None
        data_view_h[key][k] = v['Hallucination'] if 'Hallucination' in v else None

In [4]:
pd.DataFrame(data_view).to_excel("data/KoLA/results_view/model_file_dict_AR_H.xlsx")

In [5]:
pd.DataFrame(data_view_ar).to_excel("data/KoLA/results_view/model_file_AnswerRelevancy.xlsx",)

In [6]:
pd.DataFrame(data_view_h).to_excel("data/KoLA/results_view/model_file_Hallucination.xlsx",)